In [6]:
"""
word2vec 的 skip-gram 简单实现
"""

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class SkipGram(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(SkipGram, self).__init__()
        # 词嵌入层：用于获取中心词的向量
        self.u_embeddings = nn.Embedding(vocab_size, embedding_dim, sparse=True)
        # 上下文嵌入层：用于获取上下文词（正负样本）的向量
        self.v_embeddings = nn.Embedding(vocab_size, embedding_dim, sparse=True)
        self.embedding_dim = embedding_dim
        self.init_weights()

    def init_weights(self):
        # 权重初始化
        init_range = 0.5 / self.embedding_dim
        self.u_embeddings.weight.data.uniform_(-init_range, init_range)
        self.v_embeddings.weight.data.uniform_(-init_range, init_range)

    def forward(self, center_word, pos_word, neg_words):
        # 获取中心词的嵌入向量
        u_embeds = self.u_embeddings(center_word)
        # 获取正样本词的嵌入向量
        v_embeds = self.v_embeddings(pos_word)
        # 获取负样本词的嵌入向量
        neg_v_embeds = self.v_embeddings(neg_words)
        
        return u_embeds, v_embeds, neg_v_embeds

# 负采样损失函数
def negative_sampling_loss(u_embeds, v_embeds, neg_v_embeds):
    """
    u_embeds: (batch_size, embedding_size)
    v_embeds: (batch_size, embdeding_size)
    neg_v_embeds: (batch_size, num_neg_samples, embedding_size)
    """
    # 计算正样本的点积 (batch_size, 1)
    pos_score = torch.sum(torch.mul(u_embeds, v_embeds), dim=1)
    # 计算负样本的点积 (batch_size, num_neg_samples)
    neg_score = torch.sum(torch.mul(u_embeds.unsqueeze(1), neg_v_embeds), dim=2)
    
    # 将正样本的点积通过 sigmoid 转换为概率，并取 log
    pos_loss = F.logsigmoid(pos_score).squeeze()
    
    # 将负样本的点积通过 sigmoid 转换为概率，并取 log，注意要取负号
    neg_loss = torch.sum(F.logsigmoid(-neg_score), dim=1)
    
    # 结合正负样本的损失并求平均
    loss = - (pos_loss + neg_loss).mean()
    return loss


if __name__ == "__main__":
    # 假设的玩具数据
    corpus = ["hello world", "word2vec is fun", "pytorch is great"]
    vocab = sorted(list(set(" ".join(corpus).split())))
    vocab_to_int = {word: i for i, word in enumerate(vocab)}
    int_to_vocab = {i: word for i, word in enumerate(vocab)}
    vocab_size = len(vocab)
    embedding_dim = 100
    num_epochs = 100
    learning_rate = 0.01
    num_neg_samples = 5
    # 修改上下文窗口大小
    window_size = 2

    # 准备训练数据 (中心词，上下文词，负样本词)
    training_data = []
    for sentence in corpus:
        words = sentence.split()
        for i, center_word in enumerate(words):
            # 遍历上下文窗口内的词
            for j in range(1, window_size + 1):
                # 检查左侧上下文
                if i - j >= 0:
                    context_word = words[i - j]
                    # 随机采样负样本
                    neg_samples = [w.item() for w in torch.multinomial(torch.ones(vocab_size), num_neg_samples, replacement=True) if int_to_vocab[w.item()] != context_word]
                    training_data.append((vocab_to_int[center_word], vocab_to_int[context_word], neg_samples))
                # 检查右侧上下文
                if i + j < len(words):
                    context_word = words[i + j]
                    neg_samples = [w.item() for w in torch.multinomial(torch.ones(vocab_size), num_neg_samples, replacement=True) if int_to_vocab[w.item()] != context_word]
                    training_data.append((vocab_to_int[center_word], vocab_to_int[context_word], neg_samples))

    # 初始化模型和优化器
    # (模型定义和损失函数与之前相同，此处省略)
    model = SkipGram(vocab_size, embedding_dim)
    optimizer = optim.SparseAdam(model.parameters(), lr=learning_rate)

    # 训练循环
    print("开始训练...")
    for epoch in range(num_epochs):
        total_loss = 0
        for center_word_idx, pos_word_idx, neg_word_indices in training_data:
            # 将数据转换为 tensor
            center_word = torch.LongTensor([center_word_idx])
            pos_word = torch.LongTensor([pos_word_idx])
            neg_words = torch.LongTensor([neg_word_indices])

            optimizer.zero_grad()
            
            # 前向传播
            u_embeds, v_embeds, neg_v_embeds = model(center_word, pos_word, neg_words)
            
            # 计算损失
            loss = negative_sampling_loss(u_embeds, v_embeds, neg_v_embeds)
            
            # 反向传播和参数更新
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch: {epoch+1}/{num_epochs}, Loss: {total_loss/len(training_data):.4f}")

    print("\n训练完成!")
    # 训练完成后，可以获取词向量
    word_vectors = model.u_embeddings.weight.data

开始训练...
Epoch: 10/100, Loss: 1.8176
Epoch: 20/100, Loss: 1.3194
Epoch: 30/100, Loss: 1.2420
Epoch: 40/100, Loss: 1.2302
Epoch: 50/100, Loss: 1.2254
Epoch: 60/100, Loss: 1.2225
Epoch: 70/100, Loss: 1.2205
Epoch: 80/100, Loss: 1.2190
Epoch: 90/100, Loss: 1.2177
Epoch: 100/100, Loss: 1.2166

训练完成!


一个完整的深度学习模型训练过程，除了你提到的 **模型（model）**、**优化器（optimizer）**、**损失函数（loss）**、**超参数（hyperparameters）** 和 **数据（data）** 之外，通常还包含以下几个关键组件：

---

### 1. 学习率调度器（Learning Rate Scheduler）
这个组件用于在训练过程中动态调整学习率。由于在训练的不同阶段，理想的学习率是不同的（例如，开始时需要较高的学习率快速收敛，后期则需要较低的学习率来微调），学习率调度器可以自动实现这种调整，防止模型在收敛后震荡或陷入局部最优。常见的调度器包括余弦退火（Cosine Annealing）、指数衰减（Exponential Decay）和分阶段衰减（Step Decay）。

### 2. 评估指标（Metrics）
损失函数是模型训练优化的目标，而评估指标则是我们用来衡量模型性能的工具。它能更直观地反映模型在特定任务上的表现。例如，在分类任务中，除了交叉熵损失，我们还会用**准确率（Accuracy）**、**精确率（Precision）**、**召回率（Recall）**和 **F1 分数**来评估模型。

### 3. 训练循环（Training Loop）
这是将所有组件整合在一起的“引擎”。训练循环负责迭代数据、执行前向传播、计算损失、执行反向传播、更新模型参数，以及在每个周期（epoch）或批次（batch）后进行评估和记录。一个好的训练循环设计可以确保训练的稳定性和效率。

### 4. 日志与可视化（Logging and Visualization）
在训练过程中，记录关键信息（如损失、评估指标、学习率等）并将其可视化是至关重要的。这有助于研究人员和工程师实时监控训练进度，诊断问题，并比较不同实验的结果。常见的工具包括 TensorBoard、Weights & Biases 和 Comet ML。

### 5. 早停（Early Stopping）
这是一个重要的正则化技术和训练策略。它通过监控模型在验证集上的性能，当性能在连续几个周期内不再提升时，提前停止训练。这不仅可以防止模型过拟合，还能节省大量的计算资源。

---

总而言之，一个完整的深度学习项目不仅仅是搭建一个模型，更是一个系统化的工程，需要将这些组件有机地结合起来，才能高效、稳定地训练出性能优异的模型。